In [17]:
!pip install gcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602926 sha256=5e8441c49d2d1f39963817cd963fcce2bf77f399f875cccd793fc3966bb1451c
  Stored in directory: /root/.cache/pip/wheels/7c/30/88/5017af921da3a33af785f0d0fd3e944b845bc62a445a2c2f69
Successfully built gcloud


In [18]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=XMzD4bOVuekHzDMTzZwETkeLwzKq2E&prompt=consent&token_usage=remote&access_type=offline&code_challenge=wI8H5Q3ZbfMJb-n06fUOKmyAj6Ln7B5NbGUpNivUsvQ&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AcvDMrDVodKhf5PKUMX-F2Mq5vCxnFOzwtlFzW-qx7BLmee1gaGVoxlQSDlYk-zFaA8TIg

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [19]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import csv
import re

In [20]:
df = pd.read_excel('STN_qualidade_fiscal.xlsx')
df

,VA_EXERCICIO,COD_IBGE,NO_ESTADO,SG_ESTADO,POS_RANKING,VA_ACERTOS_TOTAL,PER_ACERTOS,DIM-I,DIM-II,DIM-III,DIM-IV,NO_ICF
0,2019,11,Rondônia,RO,4,63.400,0.932,14.400,28.0,9,12,B
1,2019,12,Acre,AC,10,60.800,0.894,14.800,25.0,10,11,B
2,2019,13,Amazonas,AM,6,62.600,0.920,13.600,28.0,11,10,B
3,2019,14,Roraima,RR,27,8.300,0.122,8.300,0.0,0,0,E
4,2019,15,Pará,PA,5,63.100,0.927,14.100,29.0,8,12,B
...,...,...,...,...,...,...,...,...,...,...,...,...
130,2023,43,Rio Grande do Sul,RS,15,142.579,0.932,33.579,65.0,20,24,B
131,2023,50,Mato Grosso do Sul,MS,5,150.433,0.983,35.433,66.0,22,27,A
132,2023,51,Mato Grosso,MT,13,145.868,0.953,33.868,65.0,18,29,A
133,2023,52,Goiás,GO,1,152.717,0.998,35.717,66.0,22,29,A


In [21]:
#Removendo coluna nao utilizada
df = df.drop('NO_ESTADO', axis=1)
df

,VA_EXERCICIO,COD_IBGE,SG_ESTADO,POS_RANKING,VA_ACERTOS_TOTAL,PER_ACERTOS,DIM-I,DIM-II,DIM-III,DIM-IV,NO_ICF
0,2019,11,RO,4,63.400,0.932,14.400,28.0,9,12,B
1,2019,12,AC,10,60.800,0.894,14.800,25.0,10,11,B
2,2019,13,AM,6,62.600,0.920,13.600,28.0,11,10,B
3,2019,14,RR,27,8.300,0.122,8.300,0.0,0,0,E
4,2019,15,PA,5,63.100,0.927,14.100,29.0,8,12,B
...,...,...,...,...,...,...,...,...,...,...,...
130,2023,43,RS,15,142.579,0.932,33.579,65.0,20,24,B
131,2023,50,MS,5,150.433,0.983,35.433,66.0,22,27,A
132,2023,51,MT,13,145.868,0.953,33.868,65.0,18,29,A
133,2023,52,GO,1,152.717,0.998,35.717,66.0,22,29,A


In [22]:
df = df.rename(columns={'VA_EXERCICIO':'ano','COD_IBGE':'id_uf','SG_ESTADO':'sigla_uf','POS_RANKING':'posicao_ranking','VA_ACERTOS_TOTAL':'quantidade_acertos_total', 'PER_ACERTOS':'proporcao_acertos', 'DIM-I':'dimensao_I', 'DIM-II':'dimensao_II', 'DIM-III':'dimensao_III', 'DIM-IV':'dimensao_IV', 'NO_ICF':'indicador_qualidade_informacao_contabil_fiscal'})
df.head()

,ano,id_uf,sigla_uf,posicao_ranking,quantidade_acertos_total,proporcao_acertos,dimensao_I,dimensao_II,dimensao_III,dimensao_IV,indicador_qualidade_informacao_contabil_fiscal
0,2019,11,RO,4,63.4,0.932,14.4,28.0,9,12,B
1,2019,12,AC,10,60.8,0.894,14.8,25.0,10,11,B
2,2019,13,AM,6,62.6,0.920,13.6,28.0,11,10,B
3,2019,14,RR,27,8.3,0.122,8.3,0.0,0,0,E
4,2019,15,PA,5,63.1,0.927,14.1,29.0,8,12,B


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 11 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   ano                                             135 non-null    int64  
 1   id_uf                                           135 non-null    int64  
 2   sigla_uf                                        135 non-null    object 
 3   posicao_ranking                                 135 non-null    int64  
 4   quantidade_acertos_total                        135 non-null    float64
 5   proporcao_acertos                               135 non-null    float64
 6   dimensao_I                                      135 non-null    float64
 7   dimensao_II                                     135 non-null    float64
 8   dimensao_III                                    135 non-null    int64  
 9   dimensao_IV                                

In [24]:
ordem_correta = ['ano', 'sigla_uf', 'id_uf', 'indicador_qualidade_informacao_contabil_fiscal', 'posicao_ranking', 'quantidade_acertos_total', 'proporcao_acertos', 'dimensao_I', 'dimensao_II', 'dimensao_III', 'dimensao_IV']
df = df[ordem_correta]
df

,ano,sigla_uf,id_uf,indicador_qualidade_informacao_contabil_fiscal,posicao_ranking,quantidade_acertos_total,proporcao_acertos,dimensao_I,dimensao_II,dimensao_III,dimensao_IV
0,2019,RO,11,B,4,63.400,0.932,14.400,28.0,9,12
1,2019,AC,12,B,10,60.800,0.894,14.800,25.0,10,11
2,2019,AM,13,B,6,62.600,0.920,13.600,28.0,11,10
3,2019,RR,14,E,27,8.300,0.122,8.300,0.0,0,0
4,2019,PA,15,B,5,63.100,0.927,14.100,29.0,8,12
...,...,...,...,...,...,...,...,...,...,...,...
130,2023,RS,43,B,15,142.579,0.932,33.579,65.0,20,24
131,2023,MS,50,A,5,150.433,0.983,35.433,66.0,22,27
132,2023,MT,51,A,13,145.868,0.953,33.868,65.0,18,29
133,2023,GO,52,A,1,152.717,0.998,35.717,66.0,22,29


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 11 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   ano                                             135 non-null    int64  
 1   sigla_uf                                        135 non-null    object 
 2   id_uf                                           135 non-null    int64  
 3   indicador_qualidade_informacao_contabil_fiscal  135 non-null    object 
 4   posicao_ranking                                 135 non-null    int64  
 5   quantidade_acertos_total                        135 non-null    float64
 6   proporcao_acertos                               135 non-null    float64
 7   dimensao_I                                      135 non-null    float64
 8   dimensao_II                                     135 non-null    float64
 9   dimensao_III                               

In [26]:
df['ano'].unique()

array([2019, 2020, 2021, 2022, 2023])

In [27]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referencia da informacao'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla do estado'),
 bigquery.SchemaField('id_uf','INTEGER',description='ID do estado de acordo com a base do IBGE'),
 bigquery.SchemaField('indicador_qualidade_informacao_contabil_fiscal','STRING',description='Indicador de qualidade'),
 bigquery.SchemaField('posicao_ranking','INTEGER',description='Posição no ranking de qualidade fiscal'),
 bigquery.SchemaField('quantidade_acertos_total','FLOAT',description='Quantidade de acertos'),
 bigquery.SchemaField('proporcao_acertos','FLOAT',description='Proporção de acertos'),
 bigquery.SchemaField('dimensao_I','FLOAT',description='Dimensão 1'),
 bigquery.SchemaField('dimensao_II','FLOAT',description='Dimensão 2'),
 bigquery.SchemaField('dimensao_III','INTEGER',description='Dimensão 3'),
 bigquery.SchemaField('dimensao_IV','INTEGER',description='Dimensão 4')
 ]

In [28]:
## Subindo para datalake
client = bigquery.Client(project='repositoriodedadosgpsp')
dataset_ref = client.dataset('Datalake')

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [29]:
table_ref = dataset_ref.table('STN_qualidade_fiscal_uf') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=66b08a9b-f19c-4a0e-8a72-e577bd19f50f>